In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.metrics import classification_report
import re
# Data Preprocessing
path = "train.tsv"
df = pd.read_csv(path, sep="\t") 

# remove punctuation
df['Phrase'] = df['Phrase'].str.replace(r'[^\w\s]+', '')
# remove numbers
df['Phrase'] = df['Phrase'].str.replace(r'\d+', '')
# make it all lower case
df['Phrase'] = df['Phrase'].str.lower()
# remove non-asci characters
df.Phrase.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#df['Phrase'] = df['Phrase'].str.split()

df['Tokenized_text'] = df['Phrase'].apply(word_tokenize) 

df['Sentiment']=df['Sentiment'].astype(int) #convert the star_rating column to int
df['NNLabels'] = df['Sentiment'].div(4)

df= df[df['Sentiment']!=2]

#df['label']=np.where(df['Sentiment']>=4,1,0) #1-Positve,0-Negative
# convert to NumPy Array
#train = df['Phrase'].to_numpy()
from sklearn.model_selection import train_test_split
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

X_train,X_test,Y_train, Y_test = train_test_split(df['Phrase'], df['Sentiment'], test_size=0.2, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

X_train,X_valid,Y_train, Y_valid = train_test_split(X_train,Y_train, test_size=0.5, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test: ",(X_valid.shape,Y_valid.shape))
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
tf_x_train = tfidfvectorizer.fit_transform(X_train)
tf_x_valid = tfidfvectorizer.transform(X_valid)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Train:  (61182,) (61182,) Test:  ((15296,), (15296,))
Train:  (30591,) (30591,) Test:  ((30591,), (30591,))


In [5]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

i=37

scale = StandardScaler()
X_train_normalized = scale.fit_transform(tf_x_train.toarray())
X_valid_normalized = scale.fit_transform(tf_x_valid.toarray())

kmeans = MiniBatchKMeans(n_clusters=i, random_state=10, n_init=10)
nn_X_train = kmeans.fit_transform(tf_x_train.toarray().astype(float))
preds = kmeans.fit_transform(tf_x_valid.toarray().astype(float))
nn_X_train_normalized = scale.fit_transform(nn_X_train)
nn_X_valid_normalized = scale.fit_transform(preds)
print(nn_X_valid_normalized.shape)
print(nn_X_train_normalized.shape)


(30591, 37)
(30591, 37)


In [17]:
import torch
import torch.nn as nn
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Flatten(),
      nn.Linear(37, 64),
      nn.ReLU(),
      nn.Linear(64, 128),
      nn.ReLU(),
      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Linear(64,5),
      nn.LogSoftmax(dim=1)
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)


In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [28]:


import torch
import torch.nn as nn
# Define the number of input features



Y_tensor = torch.tensor(Y_train.array).long()
X_tensor = torch.tensor(nn_X_train_normalized).float()

#X_tensor = X_tensor.to(device)

#Y_tensor = Y_tensor.to(device)

mlp = MLP()
mlp.to(device)
#print(X_tensor.shape)
# make a dataset object
dataset = torch.utils.data.TensorDataset(X_tensor, Y_tensor)

# then use it to make a dataloader object
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=4)
# make a scoring function
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    # Print epoch
    print(f'Starting epoch {epoch+1}')

      # Set current loss value
    current_loss = 0.0
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)
            
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp(inputs)
        #print(outputs.shape)
        #print(targets[4].value)
        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics

        current_loss += loss.item()
        if i % 500 == 499:
              print('Loss after mini-batch %5d: %.3f' %
                    (i + 1, current_loss / 500))
              current_loss = 0.0



#plt.scatter(trainX[:,0], trainX[:, 1], c=pred_labels)

Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20
Starting epoch 21
Starting epoch 22
Starting epoch 23
Starting epoch 24
Starting epoch 25
Starting epoch 26
Starting epoch 27
Starting epoch 28
Starting epoch 29
Starting epoch 30
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Starting epoch 41
Starting epoch 42
Starting epoch 43
Starting epoch 44
Starting epoch 45
Starting epoch 46
Starting epoch 47
Starting epoch 48
Starting epoch 49
Starting epoch 50


In [30]:
X_valid_tensor = torch.tensor(nn_X_valid_normalized).float()
preds = mlp(X_valid_tensor.to(device)).detach().numpy()
test_output_labels = np.argmax(preds,1)

report=classification_report(Y_train.array, test_output_labels.astype(int),output_dict=True)
print(report)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

# IMBD Dataset

## Just K-Means

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.stem.porter import *
nltk.download('punkt')
path = "clean_IMDBdataset.csv"
df = pd.read_csv(path, sep=",")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
X_train,X_test,Y_train, Y_test = train_test_split(df['Phrase'], df['Sentiment'], test_size=0.2, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test/Valid: ",(X_test.shape,Y_test.shape))
X_test,X_valid,Y_test, Y_valid = train_test_split(X_test,Y_test, test_size=0.5, random_state=30)
print("Test: " ,X_test.shape,Y_test.shape,"Valid: ",(X_valid.shape,Y_valid.shape))
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
tf_x_train = tfidfvectorizer.fit_transform(X_train)
tf_x_valid = tfidfvectorizer.transform(X_valid)

Train:  (40000,) (40000,) Test/Valid:  ((10000,), (10000,))
Test:  (5000,) (5000,) Valid:  ((5000,), (5000,))


In [5]:
tf_x_train.toarray()

KeyboardInterrupt: 

In [ ]:
i=2
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
scale = StandardScaler()
#X_train_normalized = scale.fit_transform(tf_x_train.toarray())
#X_valid_normalized = scale.fit_transform(tf_x_valid.toarray())

kmeans = KMeans(n_clusters=i, random_state=10, n_init=10)
nn_X_train = kmeans.fit_transform(tf_x_train.toarray().astype(float))
test_preds = kmeans.predict(tf_x_valid.toarray().astype(float))
#preds = kmeans.fit_transform(X_valid_normalized.toarray().astype(float))
#nn_X_train_normalized = scale.fit_transform(nn_X_train)
#nn_X_valid_normalized = scale.fit_transform(preds)
#print(nn_X_valid_normalized.shape)
#print(nn_X_train_normalized.shape)

In [18]:
report=classification_report(Y_valid, test_preds,output_dict=True)
report

{'0': {'precision': 0.45351473922902497,
  'recall': 0.6458557588805167,
  'f1-score': 0.532859680284192,
  'support': 929},
 '1': {'precision': 0.36730769230769234,
  'recall': 0.2089715536105033,
  'f1-score': 0.2663877266387727,
  'support': 914},
 'accuracy': 0.42919153553988065,
 'macro avg': {'precision': 0.41041121576835865,
  'recall': 0.42741365624551,
  'f1-score': 0.39962370346148235,
  'support': 1843},
 'weighted avg': {'precision': 0.4107620312061829,
  'recall': 0.42919153553988065,
  'f1-score': 0.40070809828098347,
  'support': 1843}}

## Batch-KMeans, then NLL

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.stem.porter import *
nltk.download('punkt')
path = "clean_IMDBdataset.csv"
df = pd.read_csv(path, sep=",")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
X_train,X_test,Y_train, Y_test = train_test_split(df['Phrase'], df['Sentiment'], test_size=0.2, random_state=30)
print("Train: " ,X_train.shape,Y_train.shape,"Test/Valid: ",(X_test.shape,Y_test.shape))
X_test,X_valid,Y_test, Y_valid = train_test_split(X_test,Y_test, test_size=0.5, random_state=30)
print("Test: " ,X_test.shape,Y_test.shape,"Valid: ",(X_valid.shape,Y_valid.shape))
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
tf_x_train = tfidfvectorizer.fit_transform(X_train)
tf_x_valid = tfidfvectorizer.transform(X_valid)

Train:  (40000,) (40000,) Test/Valid:  ((10000,), (10000,))
Test:  (5000,) (5000,) Valid:  ((5000,), (5000,))


In [22]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train_normalized = scale.fit_transform(tf_x_train.toarray())
X_valid_normalized = scale.fit_transform(tf_x_valid.toarray())

KeyboardInterrupt: 

In [5]:
i=37
#from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
#scale = StandardScaler()
#X_train_normalized = scale.fit_transform(tf_x_train.toarray())
#X_valid_normalized = scale.fit_transform(tf_x_valid.toarray())

kmeans = MiniBatchKMeans(n_clusters=i, random_state=10, n_init=10)
nn_X_train = kmeans.fit_transform(tf_x_train.toarray().astype(float))
#test_preds = kmeans.predict(X_valid_normalized.astype(float))
preds = kmeans.fit_transform(tf_x_valid.toarray().astype(float))


In [21]:

nn_X_train_normalized = scale.fit_transform(nn_X_train)
nn_X_valid_normalized = scale.fit_transform(preds)
print(nn_X_valid_normalized.shape)
print(nn_X_train_normalized.shape)

(5000, 37)
(40000, 37)


In [14]:
import torch
import torch.nn as nn
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Flatten(),
      nn.Linear(37, 64),
      nn.ReLU(),
      nn.Linear(64, 128),
      nn.ReLU(),
      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Linear(64,2),
      nn.LogSoftmax(dim=1)
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)


In [15]:


import torch
import torch.nn as nn
# Define the number of input features



Y_tensor = torch.tensor(Y_train.array).long()
X_tensor = torch.tensor(nn_X_train).float()

#X_tensor = X_tensor.to(device)

#Y_tensor = Y_tensor.to(device)

mlp = MLP()
mlp.to(device)
#print(X_tensor.shape)
# make a dataset object
dataset = torch.utils.data.TensorDataset(X_tensor, Y_tensor)

# then use it to make a dataloader object
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=4)
# make a scoring function
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    # Print epoch
    print(f'Starting epoch {epoch+1}')

      # Set current loss value
    current_loss = 0.0
    i=0
    for inputs, targets in trainloader:
        i+=1
        inputs, targets = inputs.to(device), targets.to(device)
            
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp(inputs)
        #print(outputs.shape)
        #print(targets[4].value)
        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics

        current_loss += loss.item()
        if i % 500 == 499:
              print('Loss after mini-batch %5d: %.3f' %
                    (i + 1, current_loss / 500))
              current_loss = 0.0



#plt.scatter(trainX[:,0], trainX[:, 1], c=pred_labels)

Starting epoch 1
Loss after mini-batch   500: 0.692
Loss after mini-batch  1000: 0.694
Loss after mini-batch  1500: 0.693
Loss after mini-batch  2000: 0.693
Loss after mini-batch  2500: 0.693
Loss after mini-batch  3000: 0.693
Loss after mini-batch  3500: 0.693
Loss after mini-batch  4000: 0.693
Starting epoch 2
Loss after mini-batch   500: 0.692
Loss after mini-batch  1000: 0.693
Loss after mini-batch  1500: 0.693
Loss after mini-batch  2000: 0.693
Loss after mini-batch  2500: 0.693
Loss after mini-batch  3000: 0.693
Loss after mini-batch  3500: 0.693
Loss after mini-batch  4000: 0.693
Starting epoch 3
Loss after mini-batch   500: 0.692
Loss after mini-batch  1000: 0.693
Loss after mini-batch  1500: 0.693
Loss after mini-batch  2000: 0.693
Loss after mini-batch  2500: 0.693
Loss after mini-batch  3000: 0.693
Loss after mini-batch  3500: 0.693
Loss after mini-batch  4000: 0.693
Starting epoch 4
Loss after mini-batch   500: 0.692
Loss after mini-batch  1000: 0.693
Loss after mini-batch 

In [16]:
from sklearn.metrics import classification_report
X_valid_tensor = torch.tensor(preds).float()
preds1 = mlp(X_valid_tensor.to(device)).cpu().detach().numpy()
test_output_labels = np.argmax(preds1,1)

report=classification_report(Y_valid.array, test_output_labels.astype(int),output_dict=True)
print(report)

{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2510}, '1': {'precision': 0.498, 'recall': 1.0, 'f1-score': 0.664886515353805, 'support': 2490}, 'accuracy': 0.498, 'macro avg': {'precision': 0.249, 'recall': 0.5, 'f1-score': 0.3324432576769025, 'support': 5000}, 'weighted avg': {'precision': 0.248004, 'recall': 0.498, 'f1-score': 0.3311134846461949, 'support': 5000}}


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1